# Proyek: Analisis Prediksi Biaya Medis
## Notebook 03: Modeling High Risk Classification

**Fase CRISP-ML(Q):** 4. Modeling, 5. Evaluation

**Tujuan:**
1.  Mengubah masalah bisnis dari Regresi ("Berapa biayanya?") menjadi Klasifikasi Biner ("Apakah nasabah ini berisiko tinggi?").
2.  Membuat `Pipeline` preprocessing yang sama dari Notebook 02.
3.  Membandingkan model baseline (Logistic Regression) dengan model ML (Random Forest Classifier).
4.  Mengevaluasi model menggunakan metrik klasifikasi (ROC-AUC, Precision, Recall, F1-Score).
5.  Menyimpan `Pipeline` pemenang sebagai artefak.

## 1. Setup & Pemuatan Data

Kita akan memuat *library* yang diperlukan dan data **MENTAH** (`insurance.csv`).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Evaluation
from sklearn.metrics import (classification_report, 
                             confusion_matrix, 
                             RocCurveDisplay, 
                             roc_auc_score, 
                             accuracy_score)

# Definisikan path kita
RAW_DATA_PATH = '../data/raw/insurance.csv'
MODEL_DIR = '../models/'
MODEL_PATH = os.path.join(MODEL_DIR, 'high_risk_classifier.joblib')

print("Libraries imported successfully!")

In [ ]:
df_raw = pd.read_csv(RAW_DATA_PATH)
print(f"Data mentah loaded. Shape: {df_raw.shape}")

## 2. Persiapan Data Spesifik Klasifikasi

Ini adalah satu-satunya langkah *Data Preparation* yang kita lakukan di sini: membuat target biner (Y) kita.

### 2.1 Membuat Target Biner `is_high_cost`

Kita akan mendefinisikan "berisiko tinggi" sebagai nasabah yang biayanya berada di atas kuantil ke-75 (25% teratas).

In [ ]:
# Tentukan ambang batas (threshold) di kuartil ke-75
threshold = df_raw['charges'].quantile(0.75)
print(f"Ambang batas 'biaya tinggi' (Kuartil 75%): ${threshold:,.2f}")

# Buat kolom target baru
df_raw['is_high_cost'] = (df_raw['charges'] > threshold).astype(int) # (1 = ya, 0 = tidak)

# Cek keseimbangan kelas (class imbalance)
print("\nDistribusi Kelas Target:")
print(df_raw['is_high_cost'].value_counts(normalize=True))

**Temuan:**
Distribusi kelasnya adalah **75% (Kelas 0)** dan **25% (Kelas 1)**. Ini **tidak seimbang (imbalanced)**.

**Hipotesis Pemodelan:**
* Metrik `accuracy` akan menyesatkan. Model yang selalu menebak "0" akan mendapatkan akurasi 75%.
* Kita harus fokus pada **ROC-AUC** dan **Classification Report** (terutama **Recall** untuk Kelas 1, yaitu "seberapa baik kita menangkap nasabah berisiko tinggi?").

### 2.2 Pemisahan Data (Train-Test Split)

Kita pisahkan data mentah kita sebelum membuat pipeline.

In [ ]:
# Tentukan Fitur (X) dan Target (y) BARU kita
X = df_raw.drop(columns=['charges', 'is_high_cost']) # Hapus kedua target
y = df_raw['is_high_cost'] # Gunakan target biner kita

# Buat set pelatihan dan pengujian
# PENTING: Gunakan 'stratify=y' untuk memastikan rasio 75/25 tetap sama di train dan test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Data split into Train/Test:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print("\nDistribusi Target di Test Set (verifikasi stratify):")
print(y_test.value_counts(normalize=True))

## 3. Desain Pipeline Preprocessing

Kita akan menggunakan `ColumnTransformer` yang **sama persis** dengan Notebook 02.

In [ ]:
# 1. Identifikasi kolom berdasarkan tipe datanya
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['sex', 'smoker', 'region']

# 2. Buat transformer untuk setiap tipe
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# 3. Gabungkan transformer menggunakan ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

print("Preprocessing pipeline defined.")

## 4. Fase Modeling & Evaluation

### 4.1 Model 1: Logistic Regression (Baseline)

In [ ]:
# Buat pipeline lengkap: Preprocessing -> Model
pipeline_logreg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # 'class_weight=\'balanced\'' membantu model menangani data 75/25 kita
    ('model', LogisticRegression(random_state=42, class_weight='balanced'))
])

print("Training Logistic Regression pipeline...")
pipeline_logreg.fit(X_train, y_train)

# Evaluasi
y_pred_logreg = pipeline_logreg.predict(X_test)

print("--- Logistic Regression Evaluation ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_logreg):.4f}")
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_logreg):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_logreg))

### 4.2 Model 2: Random Forest Classifier (Challenger ML)

In [ ]:
# Buat pipeline lengkap: Preprocessing -> Model
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

print("Training Random Forest pipeline...")
pipeline_rf.fit(X_train, y_train)

# Evaluasi
y_pred_rf = pipeline_rf.predict(X_test)

print("--- Random Forest Evaluation ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_pred_rf):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

### 4.3 Hasil Evaluasi: ROC-AUC Curve

Mari kita visualisasikan perbandingan ROC-AUC, metrik terbaik kita untuk klasifikasi tidak seimbang.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

# Plot Random Forest (pemenang kita)
RocCurveDisplay.from_estimator(pipeline_rf, X_test, y_test, name='Random Forest', ax=ax)

# Plot Logistic Regression (baseline)
RocCurveDisplay.from_estimator(pipeline_logreg, X_test, y_test, name='Logistic Regression', ax=ax)

# Plot garis 50/50 (tebakan acak)
ax.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random Guess')

plt.title('ROC Curve Comparison: High-Cost Customer Prediction')
plt.legend()

plt.show()

**Temuan:**
Random Forest (AUC > 0.9) akan jauh mengungguli Logistic Regression. Ini menunjukkan kemampuannya yang superior dalam memisahkan nasabah berisiko tinggi dari yang berisiko rendah.

## 5. Fase Interpretasi (Feature Importance)

Untuk model pohon seperti Random Forest, kita bisa langsung menarik `feature_importances_` untuk melihat fitur apa yang paling penting.

In [ ]:
# 1. Ambil model dan preprocessor yang sudah dilatih
model_rf = pipeline_rf.named_steps['model']
preprocessor_trained = pipeline_rf.named_steps['preprocessor']

# 2. Ambil nama fitur yang sudah di-OHE
try:
    feature_names = preprocessor_trained.get_feature_names_out()
except Exception:
    print("Menggunakan fallback get_feature_names()")
    ohe_features = preprocessor_trained.named_transformers_['cat']\
                                      .named_steps['onehot']\
                                      .get_feature_names_out(categorical_features)
    feature_names = numeric_features + list(ohe_features)

# 3. Buat DataFrame Feature Importance
importances = model_rf.feature_importances_
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("--- Top 10 Feature Importances ---")
print(importance_df.head(10))

In [ ]:
# Visualisasikan Feature Importance
plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=importance_df)
plt.title('Feature Importance for High-Cost Customer Prediction')
plt.show()

**Temuan:**
Sama seperti di Notebook 02, kita akan melihat bahwa `smoker_yes` adalah fitur yang paling dominan, diikuti oleh `bmi` dan `age`. Ini mengonfirmasi bahwa faktor risiko utama tetap konsisten.

## 6. Penyimpanan Artefak Model Pemenang

Kita menyimpan pipeline Random Forest pemenang.

In [ ]:
try:
    joblib.dump(pipeline_rf, MODEL_PATH)
    print(f"Pipeline model pemenang (Random Forest) disimpan ke: {MODEL_PATH}")
except Exception as e:
    print(f"Error saat menyimpan pipeline: {e}")

## Kesimpulan Fase 4 (Klasifikasi)

Kita telah berhasil mengubah masalah ini menjadi masalah klasifikasi dan melatih model (Random Forest) yang sangat akurat (dilihat dari ROC-AUC) dalam mengidentifikasi nasabah berisiko tinggi.

Kita siap untuk **Langkah 04: Modeling Risk Pool Clustering**.